<a href="https://colab.research.google.com/github/soheil-aa/Coursera_Capstone/blob/main/CapstoneProject_Week3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Week 3 Assignment**
# **Segmenting and Clustering Neighborhoods in Toronto**

## **Part 1**
## Scapring the wiki page and extarcting Canada postal codes


### Import neccessary libraries

In [14]:
# for performing your HTTP requests
import requests  

# for xml & html scrapping 
from bs4 import BeautifulSoup 

# for table analysis
import pandas as pd

#Visuals
import matplotlib.pyplot as plt

### Read the wiki and extract table data

In [2]:
# url of wikipedia page
url_canada_postal_codes = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
# Request & Response
s = requests.Session()
response = s.get(url_canada_postal_codes, timeout=10)
response # <Response [200]> means a successfull response

<Response [200]>

In [4]:
# parse response content to html
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
# get the sortale table
canada_PO_table = soup.find('table', {"class":'wikitable sortable'})

In [6]:
# tabel shape
rows = canada_PO_table.findAll("tr")
for row in rows:
    cells = row.findAll('td')

table_shape = [len(rows), len(cells)]
print("Table shape: {} x {}".format(table_shape[0], table_shape[1]))

Table shape: 181 x 3


In [7]:
# header attributes of the table
canada_PO_table_columns = [th.text.rstrip() for th in rows[0].find_all('th')]
print("column names: ", canada_PO_table_columns)

column names:  ['Postal Code', 'Borough', 'Neighbourhood']


In [8]:
# extract data
table_data = []
for row in rows[1:]:
  row_data = [d.text.rstrip() for d in row.find_all('td')]
  table_data.append(row_data)

### Create a data frame and clean and aggregate the data

In [9]:
# create a data frame
canada_PO_df = pd.DataFrame(table_data, columns = canada_PO_table_columns)
canada_PO_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [10]:
# drop rpostal codes which have no assigned borough
canada_PO_df.drop(canada_PO_df[canada_PO_df.Borough == 'Not assigned'].index, inplace=True)
canada_PO_df.reset_index(drop=True)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
# group postal codes and join the Neighborhoods
canada_PO_df = canada_PO_df.groupby(['Postal Code'], sort=False).agg( ','.join)
canada_PO_df.head()

,Borough,Neighbourhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [12]:
# if a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
idx = canada_PO_df[(canada_PO_df.Neighbourhood == 'Not assigned') & (canada_PO_df.Borough != 'Not assigned')].index
if len(idx) != 0:
  canada_PO_df.loc[idx_, 'Neighbourhood'] = canada_PO_df.loc[idx_, 'Borough']

### Results of Part 1

In [15]:
canada_PO_df.head()

,Borough,Neighbourhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [13]:
# shape of the data frame
canada_PO_df.shape

(103, 2)